In [ ]:
    !pip install geopandas
    import geopandas as gpd
    import matplotlib.pyplot
    %matplotlib inline
    import math
    import warnings
    from collections import OrderedDict

    import numpy as np
    import pandas as pd
    from shapely.geometry import LineString
    from shapely.geometry import MultiLineString
    from shapely.geometry import MultiPoint
    from shapely.geometry import MultiPolygon
    from shapely.geometry import Point
    from shapely.geometry import Polygon
    from shapely.ops import split


In [ ]:
df = gpd.read_file("/content/Kanpur_patna.shp")

In [ ]:
df.crs = {"init":"epsg:4326"}
df = df.to_crs(epsg=3857)

In [ ]:
df.length

In [ ]:
cl = gpd.read_file("/content/centerline.shp")

In [ ]:
cl.crs = {"init":"epsg:4326"}
cl = cl.to_crs(epsg=3857)

In [ ]:
ax = df.plot(color="yellow",figsize=(100,40))

cl.plot(ax=ax,color='black',figsize=(100,40))

In [ ]:
cl.to_file('fc.shp')

In [ ]:
bl = cl.boundary
bound = gpd.GeoDataFrame(geometry=cl.boundary)

In [ ]:
al = str(bound['geometry'].iloc[0])

In [ ]:
len(al)

746

In [ ]:
!pip install mplleaflet
import fiona
import numpy as np
from shapely.geometry import LineString, MultiLineString
from shapely import ops
import mplleaflet as mpl
import matplotlib.pyplot as plt

In [ ]:
mergedRoad = ops.linemerge(cl['geometry'].iloc[0])
mergedRoad.length

In [ ]:
lenSpace = np.linspace(0,mergedRoad.length,num =int(8876620))
lenSpace

In [ ]:
tempList = []
for space in lenSpace:
    tempPoint =mergedRoad.interpolate(space)
    tempList.append(tempPoint)

denseRoad = LineString(tempList)
display(denseRoad)

In [ ]:
lines = []
total = 0.0
ftotal = 0
x,y = denseRoad.coords.xy
cd_length =3000
start = 0
for i in range(0,len(x)-1):
  a =(x[i],y[i])
  b =(x[i+1],y[i+1])
  AB = LineString([a, b])
  total = total+ AB.length
  nt = total
  if(total>5000):
    print (total)
    end=i+1
    c=(x[start],y[start])
    d=(x[end],y[end])
    ab = LineString([c,d]);
    left = ab.parallel_offset(cd_length / 2, 'left')
    right = ab.parallel_offset(cd_length / 2, 'right')
    c = left.boundary[1]
    d = right.boundary[0]  # note the different orientation for right offset
    cd = LineString([c, d])
    lines.append(cd)
    start = end
    total = 0


In [ ]:
    import math
    import warnings
    from collections import OrderedDict

    import numpy as np
    import pandas as pd
    from shapely.geometry import LineString
    from shapely.geometry import MultiLineString
    from shapely.geometry import MultiPoint
    from shapely.geometry import MultiPolygon
    from shapely.geometry import Point
    from shapely.geometry import Polygon
    from shapely.ops import split

    for line in lines:
        df['geometry'].iloc[0] = MultiPolygon(split(df['geometry'].iloc[0], line))

In [ ]:
df.plot(figsize=(100,40),color="rgb")

In [ ]:
df.head()

In [ ]:
df.to_file('segment.shp')